In [1]:
import os
os.environ['MLFLOW_TRACKING_URI'] = 'https://dagshub.com/Rajusubba/datascienceproject.mlflow'
os.environ['MLFLOW_TRACKING_USERNAME'] = 'Rajusubba'
os.environ['MLFLOW_TRACKING_PASSWORD'] = 'b0503c19e8390be0103da55f59ac7f191dc0d676'

os.chdir("/Users/rajusubba/Documents/MLOpsProject/datascienceproject")

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_url: str
    
    

In [3]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories, save_json

In [4]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath: Path = CONFIG_FILE_PATH,
        params_filepath: Path = PARAMS_FILE_PATH,
        schema_filepath: Path = SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
                           
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            test_data_path = config.test_data_path,
            model_path = config.model_path,
            all_params = params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_url = "https://dagshub.com/Rajusubba/datascienceproject.mlflow"
        )
        return model_evaluation_config
            
   
    

In [5]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [12]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
        
    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
        
    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
        
        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]
        
        mlflow.set_tracking_uri(self.config.mlflow_url)
        
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            predicted_qualities = model.predict(test_x)
            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)
            mlflow.set_tracking_uri(self.config.mlflow_url)
            mlflow.log_params(self.config.all_params)
            
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2", r2)
            
            '''if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticNetModel")
            else:
                mlflow.sklearn.log_model(model, "model")'''
                
            mlflow.log_artifact(str(self.config.model_path), artifact_path="model")

In [13]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2025-11-20 11:45:16,813: INFO: common: yaml file: config/config.yaml loaded successfully
[2025-11-20 11:45:16,815: INFO: common: yaml file: params.yaml loaded successfully
[2025-11-20 11:45:16,817: INFO: common: yaml file: schema.yaml loaded successfully
[2025-11-20 11:45:16,818: INFO: common: Created directory at: artifacts
[2025-11-20 11:45:16,818: INFO: common: Created directory at: artifacts/model_evaluation
[2025-11-20 11:45:17,360: INFO: common: JSON file saved at: artifacts/model_evaluation/metric.json
🏃 View run traveling-boar-833 at: https://dagshub.com/Rajusubba/datascienceproject.mlflow/#/experiments/0/runs/9bd4c5246af54963991fd2a5aa82b6f4
🧪 View experiment at: https://dagshub.com/Rajusubba/datascienceproject.mlflow/#/experiments/0
